In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./medical.csv")

In [3]:
df

,symptoms,triage
0,Severe chest pain with shortness of breath,Emergency
1,Sudden weakness on one side of the body,Emergency
2,Severe difficulty breathing,Emergency
3,Uncontrollable bleeding,Emergency
4,Loss of consciousness,Emergency
...,...,...
90,Chest discomfort while walking,Urgent
91,Moderate dehydration symptoms,Urgent
92,Fever with chills for two days,Urgent
93,Persistent joint stiffness,Urgent


In [4]:
l=[str(df.iloc[i]) for i in range(len(df))]

In [5]:
from langchain_core.documents import Document
doc=[Document(page_content=text) for text in l]

In [6]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vectorstore = Chroma.from_documents(
    documents=doc,
    embedding=embedding_model,
    persist_directory="vector_db"
)


C:\Users\jkamisex\AppData\Local\Temp\ipykernel_22096\2681986837.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
C:\Users\jkamisex\OneDrive - Intel Corporation\Desktop\ai_friday\medical\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [8]:
from langchain_classic.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_medical",
    "Search and return relavent information.",
)

In [9]:
print(retriever_tool.invoke({"query": "how prime minister of india"}))

symptoms    Red swollen insect bite
triage                       Urgent
Name: 34, dtype: object

symptoms    Red swollen insect bite
triage                       Urgent
Name: 34, dtype: object

symptoms    Red swollen insect bite
triage                       Urgent
Name: 34, dtype: object

symptoms    Red swollen insect bite
triage                       Urgent
Name: 34, dtype: object

symptoms    Red swollen insect bite
triage                       Urgent
Name: 34, dtype: object


In [10]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['GROQ_API_KEY']="API_KEY"

In [11]:
from langchain_groq import ChatGroq

response_model = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
    max_tokens=None,
    # reasoning_format="parsed", # not a thinking model
    timeout=None,
    max_retries=2,
    # other params...
)

In [12]:
from langchain_core.prompts import PromptTemplate

# Define the template string with a placeholder for 'topic'
template_string = '''You are a helpful assistant, Suggesting required triage based on the users question and provided context.
If their is no relation between question and provided context, please reply as you don't know.

{context}

Question:{question}
Answer:
'''
# Create a PromptTemplate instance
prompt_template = PromptTemplate(
    template=template_string,
    input_variables=["contex,question"]
)

# Format the prompt with a specific value for 'topic'
inp=input("please ask your question: ")
formatted_prompt = prompt_template.format(context=retriever_tool.invoke({"query": inp}),question=inp)

# Print the resulting prompt
# print(formatted_prompt)

please ask your question:  i am feeling drowsy and i didnt slept for 3days 


In [13]:
out=response_model.invoke(formatted_prompt)
out.content

'Routine'